In [4]:
import numpy as np

camera_normal = np.zeros([3, 2, 2], dtype=np.float32)
camera_normal[:,0,0] = 1
camera_normal[:,0,1] = 2
camera_normal[:,1,0] = 3
camera_normal[:,1,1] = 4

print(camera_normal)
print(camera_normal[:,1,1])


[[[1. 2.]
  [3. 4.]]

 [[1. 2.]
  [3. 4.]]

 [[1. 2.]
  [3. 4.]]]
[4. 4. 4.]


In [5]:
camera_normal = np.zeros([2, 2, 3], dtype=np.float32)
camera_normal[0,0,:] = 1
camera_normal[0,1,:] = 2
camera_normal[1,0,:] = 3
camera_normal[1,1,:] = 4

print(camera_normal)
print(camera_normal[1,1,:])

[[[1. 1. 1.]
  [2. 2. 2.]]

 [[3. 3. 3.]
  [4. 4. 4.]]]
[4. 4. 4.]


In [6]:
import numpy as np
from matplotlib import pyplot as plt
import torch
from torchvision import transforms
from data_loader import Dataset,Options

# from skimage.transform import resize
import skimage


exr_arr = np.load('data/surface-normals-converted/000000000-normals.exr.npy')
exr_arr[exr_arr>1] = 1
exr_arr[exr_arr<-1] = -1

print('orig surface normals shape:')
print(exr_arr.shape)
exr_arr = exr_arr.transpose(1,2,0)
plt.imshow(exr_arr)
plt.show()

# Resize the surface normals
exr_arr_cropped = skimage.transform.resize(exr_arr, (244, 244))
plt.imshow(exr_arr_cropped)
plt.show()

ImportError: cannot import name 'Options'

## Test Edge Detection on Depth

In [ ]:
%matplotlib inline 

import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from data_loader import Dataset,Options
import OpenEXR, Imath
from scipy.misc import imsave

class OPT():
    def __init__(self):
        self.dataroot = './data/'
        self.file_list = './data/datalist'
        self.batchSize = 32
        self.shuffle = True
        self.phase = 'train'
        self.num_epochs = 500
        self.imsize = 224
        self.num_classes = int(3)
        self.gpu = '0'
        self.logs_path = 'logs/exp9'
        self.use_pretrained = False

opt = OPT()

###################### DataLoader #############################
dataloader = Dataset(opt)


depth_path = '/media/shrek/work/datasets/google-warehouse/greppy-surface-normals-bottle-1500/dataset/%09d-depth.exr'
edges_output_path = '/media/shrek/work/datasets/google-warehouse/greppy-surface-normals-bottle-1500/edges-depth-imgs/%09d-depth-edges.jpg'

for i in range (1,100):
    # Load Depth Img and Apply Blur
    print('Loading img %d'%(i))
    depth_img_orig = dataloader.exr_loader(depth_path%(i), ndim=1)
    depth_img_orig = cv2.GaussianBlur(depth_img_orig,(5,5),0)
    
    # Make all depth values greater than 2.5m as 0 (for masking edge matrix)
    depth_img_mask = depth_img_orig.copy()
    depth_img_mask[depth_img_mask > 2.5]=0
    depth_img_mask[depth_img_mask > 0 ]=1
    
    # Apply Laplacian filters for edge detection
    edges_lap = cv2.Laplacian(depth_img_orig, cv2.CV_8U, ksize=7, borderType=0 )
    edges_lap_binary = edges_lap.copy()
    print(edges_lap_binary.max())
    edges_lap_binary[edges_lap_binary>1] = 255
    edges_lap_binary[edges_lap_binary<=1] = 0
    edges_lap_binary = edges_lap_binary * depth_img_mask
    
    # Apply Canny filter
    depth_img_masked = depth_img_orig * depth_img_mask
    depth_img_masked *= 255.0/depth_img_masked.max()
    depth_img_masked = np.uint8(depth_img_masked)
    edges = cv2.Canny(depth_img_masked,5,10)
    
    display_output = 1
    if(display_output):
        fig1 = plt.figure(figsize=(12,12))
        plt.imshow(depth_img_orig, cmap='gray')
        plt.show()
        fig2 = plt.figure(figsize=(12,12))
        plt.imshow(edges_lap, cmap='gray')
        plt.show()
        fig3 = plt.figure(figsize=(12,12))
        plt.imshow(edges_lap_binary, cmap='gray')
        plt.show()
#         fig4 = plt.figure(figsize=(12,12))
#         plt.imshow(edges, cmap='gray')
#         plt.show()
    

    save_output = True
    if(save_output):
        imsave( edges_output_path%(i), edges_lap_binary )



# Test Edge Detection on Surface Normals

In [ ]:
%matplotlib inline 

import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from data_loader import Dataset,Options
import OpenEXR, Imath
from scipy.misc import imsave
import imageio

class OPT():
    def __init__(self):
        self.dataroot = './data/'
        self.file_list = './data/datalist'
        self.batchSize = 32
        self.shuffle = True
        self.phase = 'train'
        self.num_epochs = 500
        self.imsize = 224
        self.num_classes = int(3)
        self.gpu = '0'
        self.logs_path = 'logs/exp9'
        self.use_pretrained = False

opt = OPT()

###################### DataLoader #############################
dataloader = Dataset(opt)

depth_path = '/media/shrek/work/datasets/google-warehouse/test-edges/preprocessed-camera-normals/%09d-cameraNormals.npy'
depth_path_rgb = '/media/shrek/work/datasets/google-warehouse/test-edges/preprocessed-camera-normals/rgb-visualizations/%09d-cameraNormals.png'
edges_output_path = '/media/shrek/work/datasets/google-warehouse/test-edges/preprocessed-camera-normals/edges-normals-imgs/%09d-normals-edges.jpg'

for i in range (1,100):
    # Load Depth Img and Apply Blur
    print('Loading img %d'%(i))
    #     depth_img_orig = np.load(depth_path%(i)).transpose(1,2,0)
    depth_img_orig_rgb = imageio.imread(depth_path_rgb%(i))
    depth_img_orig = cv2.cvtColor(depth_img_orig_rgb, cv2.COLOR_BGR2GRAY)
    depth_img_orig = cv2.GaussianBlur(depth_img_orig,(5,5),0)
    
#     # Make all depth values greater than 2.5m as 0 (for masking edge matrix)
#     depth_img_mask = depth_img_orig.copy()
#     depth_img_mask[depth_img_mask > 2.5]=0
#     depth_img_mask[depth_img_mask > 0 ]=1
    
    # Apply Laplacian filters for edge detection
    edges_lap = cv2.Laplacian(depth_img_orig, cv2.CV_8U, ksize=3, borderType=1 )
    edges_lap_binary = edges_lap.copy()
    print(edges_lap_binary.max())
    edges_lap_binary[edges_lap_binary>15] = 255
    edges_lap_binary[edges_lap_binary<=10] = 0
#     edges_lap_binary = edges_lap_binary * depth_img_mask
    
    # Apply Canny filter
#     depth_img_masked = depth_img_orig * depth_img_mask
#     depth_img_masked *= 255.0/depth_img_masked.max()
#     depth_img_masked = np.uint8(depth_img_masked)
    edges = cv2.Canny(depth_img_orig,10,40)
    
    display_output = 1
    if(display_output):
        fig1 = plt.figure(figsize=(12,12))
        plt.imshow(depth_img_orig_rgb)
        plt.show()
        fig2 = plt.figure(figsize=(12,12))
        plt.imshow(edges_lap, cmap='gray')
        plt.show()
        fig3 = plt.figure(figsize=(12,12))
        plt.imshow(edges_lap_binary, cmap='gray')
        plt.show()
        fig4 = plt.figure(figsize=(12,12))
        plt.imshow(edges, cmap='gray')
        plt.show()
    

    save_output = True
    if(save_output):
        imsave( edges_output_path%(i), edges_lap_binary )



In [10]:
import torch
import torch.nn as nn

loss = nn.CrossEntropyLoss()

input = torch.randn(3, 5, requires_grad=True)
print(input)
print(input.shape)

target = torch.empty(3, dtype=torch.long).random_(5)
print(target)
print(target.shape)

output = loss(input, target)
print(output)


output.backward()

tensor([[-0.7403, -0.3435,  0.1837,  0.9554,  0.0386],
        [-1.8947,  0.1641,  0.3512,  0.6326,  0.3800],
        [ 0.1681,  0.9758,  1.0716, -0.1566,  0.2910]], requires_grad=True)
torch.Size([3, 5])
tensor([4, 1, 4])
torch.Size([3])
tensor(1.7672, grad_fn=<NllLossBackward>)


In [17]:
import imageio

label_path = '/media/shrek/work/datasets/public/shuran-mini-pytorch-examples/nyuv2_test_class13/new_nyu_class13_0001.png'

label = imageio.imread(label_path)

print(label.shape)
print(label.dtype)


(480, 640)
uint8


# Test the Pixel Values within Original depth2depth Weights examples

In [3]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from PIL import Image 
import imageio
from matplotlib import interactive
interactive(True)
%matplotlib qt  

# read in outlines/weights file
path_outlines = '/home/shrek/greppy/brain/transparent_detection/models/test_depth2depth/deepcompletion/data/bound_realsense_test_bound/realsense_004_bound_est.png'
path_weights = '/home/shrek/greppy/brain/transparent_detection/models/test_depth2depth/deepcompletion/data/bound_realsense_weight/realsense_004_weight.png'

outlines = Image.open(path_outlines)
outlines = np.array(outlines)
weights = Image.open(path_weights)
weights = np.array(weights)



# Create interactive plot to analyze pixel values
fig = plt.figure(figsize=(8, 6))
ax0 = plt.subplot(121)
ax1 = plt.subplot(122)


ax0.imshow(outlines)
ax0.set_title('Predicted Outlines')
ax1.imshow(weights)
ax1.set_title('Weights from \n Outlines')
    
plt.show()


    

# Create weights and save

In [ ]:
import imageio
import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from PIL import Image


img_path = '/home/shrek/greppy/brain/transparent_detection/models/DeepCompletionRelease/torch/result/bound_realsense_test_bound/realsense_004_bound_est.png'

# brain/surface-normals/references/DeepCompletionRelease/torch/result/bound_realsense_test_bound/realsense_004_bound_est.png

file = Image.open(img_path)
file_arr = np.array(file).astype(np.float64)/255
weight = (1-file_arr[:,:,1])
x = np.power(weight,3)
x = np.multiply(x,1000)

final_weight = x.astype(np.uint16)
print(final_weight.shape)
print(final_weight)

# imageio.imwrite('weights.png', final_weight)
array_buffer = final_weight.tobytes()
img = Image.new("I", final_weight.T.shape)
img.frombytes(array_buffer, 'raw', 'I;16')
img.save('weights_2.png')

# imwrite(uint16(weight*1000), fullfile(output_dir, strrep(files(a).name, '_bound_est.png', '_weight.png')))



# x = np.array([1,2,3])
# y = np.array([1,2,3])

# z = np.multiply(x,y)
# print(z)
# q = np.multiply(z,x)
# print(q)


path_img_test = '/home/shrek/greppy/brain/transparent_detection/models/DeepCompletionRelease/torch/result/bound_realsense_weight/realsense_004_weight.png'
# path_img_train = 'data/train/edges-imgs/000000000-segmentation.png'

weight = Image.open(path_img_test)
weight = np.array(weight)
print(weight)

# weight = Image.open(path_img_train)
# weight = np.array(weight)
# print(weight)